In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [17]:
# datasets

categorical_data = pd.read_excel("train_new/TRAIN_CATEGORICAL_METADATA_new.xlsx")
quantitative_data = pd.read_excel("train_new/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
connectome_data = pd.read_csv("train_new/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
train_data = pd.read_excel("train_new/TRAINING_SOLUTIONS.xlsx")

print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   participant_id  1213 non-null   object
 1   ADHD_Outcome    1213 non-null   int64 
 2   Sex_F           1213 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 28.6+ KB
None


In [14]:
merged_data = categorical_data.merge(quantitative_data, on='participant_id', how='inner')
merged_data = merged_data.merge(connectome_data, on='participant_id', how='inner')
merged_data = merged_data.merge(train_data, on='participant_id', how='inner')

print(merged_data)
print("Shape after dropping missing values:", merged_data.shape)


     participant_id  Basic_Demos_Enroll_Year  Basic_Demos_Study_Site  \
0      00aIpNTbG5uh                     2019                       4   
1      00fV0OyyoLfw                     2017                       1   
2      04X1eiS79T4B                     2017                       1   
3      05ocQutkURd6                     2018                       1   
4      06YUNBA9ZRLq                     2018                       1   
...             ...                      ...                     ...   
1208   zwjJWCRzKhDz                     2019                       4   
1209   zwXD5v17Rx01                     2018                       1   
1210   zWzLCi3NTBTd                     2018                       3   
1211   Zy9GTHDxUbXU                     2019                       4   
1212   Zye7yYRQohXi                     2017                       1   

      PreInt_Demos_Fam_Child_Ethnicity  PreInt_Demos_Fam_Child_Race  \
0                                  1.0                          

In [ ]:
# missing values
merged_data.isnull().sum()

merged_data_clean = merged_data.dropna()

# merged_data_clean.isnull().sum()
print("Shape after dropping missing values:", merged_data_clean.shape)



Shape after dropping missing values: (619, 19930)


In [16]:
categorical_cols = merged_data_clean.select_dtypes(include='object').columns
print(categorical_cols)

Index(['participant_id'], dtype='object')


In [20]:
# target variables
y = merged_data_clean[['Sex_F', 'ADHD_Outcome']]

# dropping participant_id and target variables
X = merged_data_clean.drop(['participant_id', 'Sex_F', 'ADHD_Outcome'], axis=1)


In [24]:

X_scaled = StandardScaler().fit_transform(X)


In [23]:
# train - test split data

X_train, X_test, y_train, y_test = train_test_split( X_scaled, y, test_size=0.2, random_state=42 )
